# Recommender System
### Collaborative filtering using Matrix Factorization for implicit feedback data using simple SVD largest k Singular values.
The matrix factorization performed in this notebook Computes the largest k singular values/vectors for a sparse matrix. Based upon the Largest K singular Values we find top K Recommended item. We are using scipy.sparse.linalg library which implements SVD using ARPACK as an eigensolver on A.H * A or A * A.H, depending on which one is more efficient.
Note: Datafiles are built from scratch in this notebook only if they don't exist on disk. However, to force rebuild any datafile, there will be a REBUILD_* constant in the respective cell that should be set to True

In [1]:
### Imports
import pandas as pd
import numpy as np
import sys
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg
from scipy.sparse import coo_matrix, csr_matrix
from numpy import bincount, log, sqrt
import itertools
import time
from pathlib import Path

# Load datasets

In [2]:
# Setting paths for data files
base_path="../data/"
product_user_matrix_path=base_path+"product_user_matrix.npz"
order_products_prior_path=base_path+"../data/order_products__prior.csv"
order_products_train_path=base_path+"../data/order_products__train.csv"
orders_path=base_path+"../data/orders.csv"
products_path=base_path+"../data/products.csv"
test_data_path = base_path+'user_products__test.csv'
matrix_df_path = base_path+"user_products__prior.csv"
matrix_path = base_path+"product_user_matrix.npz"
product_factor_50_path= base_path+"product_factor_50.npy"
user_factor_50_path= base_path+"user_factor_50.npy"
product_factor_100_path= base_path+"product_factor_100.npy"
user_factor_100_path= base_path+"user_factor_100.npy"

In [3]:
# Order & User Datasets
df_order_products_prior = pd.read_csv(order_products_prior_path)
df_order_products_train = pd.read_csv(order_products_train_path)
df_orders = pd.read_csv(orders_path) 

# Products
df_products = pd.read_csv(products_path)

# No. of items to recommend
rec_items = 10

# Making Test Data

In [4]:
def make_test_data(filepath, df_orders, df_order_products_train):
    """
    Generates the test dataset and saves it to disk at the given path
    """
    
    start = time.time()
    print("Creating test data ...")

    # Read train csv
    df_order_user_current = df_orders.loc[(df_orders.eval_set == "train")].reset_index()
    df_order_user_current = df_order_user_current[["order_id", "user_id"]]
    
    print(df_order_user_current.head())
    
    # Sanity check #1: `current_order_user_df` and `df_order_products_train` should have the same number of 
    # unique order ids
    assert len(df_order_user_current["order_id"].unique()) == len(df_order_products_train["order_id"].unique())

    # Convert train dataframe to a similar format
    df_order_products_test = df_order_products_train[["order_id", "product_id"]]
    df_order_products_test = df_order_products_test.groupby("order_id")["product_id"].apply(list).reset_index().rename(columns={"product_id": "products"})
    print(df_order_products_test.head())
        
    # Sanity check #2: `df_order_products_test` and `df_order_user_current` should have the same number of 
    # records before attempting to merge them
    assert df_order_products_test.size == df_order_user_current.size

    # Merge on order id
    df_user_products_test = pd.merge(df_order_user_current, df_order_products_test, on="order_id")
    df_user_products_test = df_user_products_test[["user_id", "products"]]

    # Write to disk
    df_user_products_test.to_csv(filepath, index_label=False, index=False)
    
    print("Completed in {:.2f}s".format(time.time() - start))


# Generate test data if it doesn't exist already
REBUILD_TEST_DATA = True
if REBUILD_TEST_DATA or not Path(test_data_path).is_file():
    make_test_data(test_data_path, df_orders, df_order_products_train)

df_user_products_test = pd.read_csv(test_data_path)

Creating test data ...
   order_id  user_id
0         1        1
1        13        3
2        26        5
3        36        6
4        44        7
   order_id products
0         1      [1]
1        13      [9]
2        26     [20]
3        36     [26]
4        44     [35]
Completed in 2.81s


# Utility Matrix

#### Making Dataframe for user-products-quantity for order_prior

In [5]:
def get_user_product_prior_df(filepath, df_orders, df_order_products_prior):
    
    """
    Generates a dataframe of users and their prior products purchases, and writes it to disk at the given path
    """
    
    start = time.time()
    print("Creating prior user product data frame ...")

    # Consider ony "prior" orders and remove all columns except `user_id` from `df_orders`
    df_order_user_prior = df_orders.loc[df_orders.eval_set == "prior"]
    df_order_user_prior = df_order_user_prior[["order_id", "user_id"]]
    
    # Remove all columns except order_id and user_id from df_orders and 
    # merge the above on `order_id` and remove `order_id`
    df_merged = pd.merge(df_order_user_prior, df_order_products_prior[["order_id", "product_id"]], on="order_id")
    df_user_product_prior = df_merged[["user_id", "product_id"]]
    df_user_product_prior = df_user_product_prior.groupby(["user_id", "product_id"]).size().reset_index().rename(columns={0:"quantity"})
    
    # Write to disk
    df_user_product_prior.to_csv(filepath, index_label=False, index=False)

    print("Completed in {:.2f}s".format(time.time() - start))


# Build dataframe of users, products and quantity bought using prior datasets
REBUILD_MATRIX_DF = True
if REBUILD_MATRIX_DF or not Path(matrix_df_path).is_file():
    get_user_product_prior_df(matrix_df_path, df_orders, df_order_products_prior)
df_user_product_prior = pd.read_csv(matrix_df_path)
# Making them as category for making dictonary of user and item ids later for easy 
# mapping from sparse Matrix representation
df_user_product_prior["user_id"] = df_user_product_prior["user_id"].astype("category")
df_user_product_prior["product_id"] = df_user_product_prior["product_id"].astype("category")
df_user_product_prior.head()

Creating prior user product data frame ...
Completed in 2.00s


,user_id,product_id,quantity
0,1,2,1
1,1,3,1
2,1,137756,1
3,1,152486,1
4,2,5,1


### User-Item Matrix

In [6]:
def build_product_user_matrix(matrix_path, df_user_product_prior):
    """
    Generates a utility matrix representing purchase history of users, and writes it to disk.
    Rows and Columns represent products and users respectively.
    """
    start = time.time()
    print("Creating product user matrix ...")

    product_user_matrix = sparse.coo_matrix((df_user_product_prior["quantity"],
                                            (df_user_product_prior["product_id"].cat.codes.copy(),
                                             df_user_product_prior["user_id"].cat.codes.copy())))    
    sparse.save_npz(matrix_path, product_user_matrix)
    
    print("Completed in {:.2f}s".format(time.time() - start))

In [7]:
# Build dataframe of users, products and quantity bought using prior datasets
REBUILD_USER_MATRIX_DF = True
if REBUILD_USER_MATRIX_DF or not Path(matrix_path).is_file():
    build_product_user_matrix(matrix_path, df_user_product_prior)    
product_user_matrix=sparse.load_npz(product_user_matrix_path).tocsr().astype(np.float32)

product_user_matrix

Creating product user matrix ...
Completed in 0.26s


<142481x45184 sparse matrix of type '<class 'numpy.float32'>'
	with 286402 stored elements in Compressed Sparse Row format>

### Sparsity Check

In [8]:

# How sparse is the utility matrix?
def sparsity(matrix):
    total_size = matrix.shape[0] * matrix.shape[1]
    actual_size = matrix.size
    sparsity = (1 - (actual_size / total_size)) * 100
    return(sparsity)

sparsity(product_user_matrix)

99.99555128671498

# SVD based Model

### Calculating User and product factors

#### Factors= 50

In [9]:
# product_factors_50,user_factors_50 here denote 50 latent Factors considered
REBUILD_FACTORS= True
if REBUILD_FACTORS or not ((Path(product_factor_50_path)).is_file() 
                           and (Path(user_factor_50_path)).is_file()): 
    #Calculating the product and user factors
    product_factors_50, S, user_factors_50 = linalg.svds(product_user_matrix, 50)
    # changing to user* factor format
    user_factors_50=user_factors_50.T*S
    # saving the user and product factors
    np.save(product_factor_50_path, product_factors_50)
    np.save(user_factor_50_path, user_factors_50)
else:
    # Loading the user and product factors 
    product_factors_50=np.load(product_factor_50_path)
    user_factors_50=np.load(user_factor_50_path)    

#### Factors= 100

In [10]:
# product_factors_100,user_factors_100 here denotes 100 latent Factors considered
REBUILD_FACTORS= True
if REBUILD_FACTORS or not ((Path(product_factor_100_path)).is_file() 
                           and (Path(user_factor_100_path)).is_file()): 
    #Calculating the product and user factors
    product_factors_100, S, user_factors_100 = linalg.svds(product_user_matrix, 100)
    # changing to user* factor format
    user_factors_100=user_factors_100.T*S
    # saving the user and product factors
    np.save(product_factor_100_path, product_factors_100)
    np.save(user_factor_100_path, user_factors_100)
else:
    # Loading the user and product factors 
    product_factors_100=np.load(product_factor_100_path)
    user_factors_100=np.load(user_factor_100_path)    

In [11]:
# Class To find the top recommended items given a user_id
class TopRecommended(object):
    def __init__(self, product_factors,user_factors,product_user_matrix):
        self.product_factors =product_factors
        self.user_factors =user_factors
        self.product_user_matrix=product_user_matrix
    def recommend(self, user_id, N=rec_items):
        
        """
        Finds top K Recommendations
        """
        scores =  self.user_factors[user_id].dot(self.product_factors.T)
        best = np.argpartition(scores, -N)[-N:]
        return sorted(zip(best, scores[best]), key=lambda x: -x[1])

    def recommend_new(self, user_id, N=rec_items):
        """
        Finds Top k new Recommendations
        """
        scores =  self.user_factors[user_id].dot(self.product_factors.T)
        bought_indices=product_user_matrix.T[user_id].nonzero()[1]
        count = N + len(bought_indices)
        ids = np.argpartition(scores, -count)[-count:]
        best = sorted(zip(ids, scores[ids]), key=lambda x: -x[1])        
        return list(itertools.islice((rec for rec in best if rec[0] not in bought_indices), N))  

# Example Recommendations

### Dictonary to map User_id & Product_id in Utility Matrix

In [12]:
# Since the utility matrix is 0-indexed, the below dict is required to convert between `ids` and `indices`.
# For example, `product_id` 1 in the dataset is represented by the `0`th row of the utility matrix.

# Maps user_id: user index
u_dict = {uid:i for i, uid in enumerate(df_user_product_prior["user_id"].cat.categories)}

# Maps product_index: product id
p_dict = dict(enumerate(df_user_product_prior["product_id"].cat.categories))

In [13]:
# Initializing class for factors 50 which returns top recommended items for a user_id
svd_recm=TopRecommended(product_factors_50,user_factors_50,product_user_matrix)

# Initializing class for factors 100 which returns top recommended items for a user_id
svd_recm_100=TopRecommended(product_factors_100,user_factors_100,product_user_matrix)


In [14]:
# # Recommend items for a user 1
# user_id = 1
# print("User ID :",user_id)
# # New Recommendations and Old Recommendations
# recommendations_all = svd_recm.recommend(u_dict[user_id],N=rec_items)
# recommendations_new = svd_recm.recommend_new(u_dict[user_id],N=rec_items)

### Actual Products Bought

In [15]:
users = pd.read_csv("../Data/users.csv")

col = ['model','reviewerID','act_products','act_imurl','rec_products','rec_imurl']
model='svd'
svd_rec_items = pd.DataFrame(columns=col)

for r, value in users.iterrows():
    act_products = []
    act_imurl =[]
    rec_products = []
    rec_imurl= []
    
    user_id = value[0]

    recommendations_new = svd_recm.recommend_new(u_dict[user_id],N=rec_items)   
    row = df_user_products_test.loc[df_user_products_test.user_id == user_id]
    actual = list(row["products"])

    actual = actual[0][1:-1]
    actual = list(np.array([p.strip() for p in actual.strip().split(",")]).astype(np.int64))
    act_products = []
    for pid in actual:
#         act_products.extend((df_products.loc[df_products.product_id == pid].product_name).tolist())
        act_products.extend((df_products.loc[df_products.product_id == pid].product_id).tolist())
        act_imurl.extend((df_products.loc[df_products.product_id == pid].imUrl).tolist())
    print("Actual products bought by user {}\n{}\n\n".format(user_id, act_products))

    # New Products Recommended 
    rec_products=[]
    for recommend in recommendations_new:
#         rec_products.extend((df_products.loc[df_products.product_id == p_dict[recommend[0]]].product_name).tolist())
        rec_products.extend((df_products.loc[df_products.product_id == p_dict[recommend[0]]].product_id).tolist())
        rec_imurl.extend((df_products.loc[df_products.product_id == p_dict[recommend[0]]].imUrl).tolist())
    print("New products recommended to user {}\n{}".format(user_id, rec_products))


    svd_rec_items.loc[len(svd_rec_items)] = [model,user_id,act_products,act_imurl,rec_products,rec_imurl]

    svd_rec_items.head()

Actual products bought by user 34418
[9996]


New products recommended to user 34418
[1696, 2571, 647, 1646, 3882, 1919, 3379, 1928, 2451, 363]
Actual products bought by user 17284
[9993]


New products recommended to user 17284
[1919, 272, 1397, 839, 1921, 700, 6769, 5325, 5327, 4701]
Actual products bought by user 8132
[12286]


New products recommended to user 8132
[11737, 1919, 1397, 889, 839, 272, 4567, 193, 1862, 1959]
Actual products bought by user 36218
[194]


New products recommended to user 36218
[10000, 194, 647, 1649, 378, 794, 791, 1862, 4091, 14897]
Actual products bought by user 9127
[1804]


New products recommended to user 9127
[700, 1804, 1919, 601, 5326, 4595, 11798, 1397, 14071, 22964]
Actual products bought by user 41538
[3831]


New products recommended to user 41538
[3831, 2529, 1959, 1646, 11305, 4112, 4381, 14828, 4609, 2853]
Actual products bought by user 7356
[4619]


New products recommended to user 7356
[1397, 839, 272, 1862, 3831, 700, 6769, 23978, 1917, 

In [16]:
svd_rec_items.to_csv('../data/svd_rec_items.csv', index=False)

In [17]:
# # Actual
# row = df_user_products_test.loc[df_user_products_test.user_id == user_id]
# actual = list(row["products"])

# actual = actual[0][1:-1]
# actual = list(np.array([p.strip() for p in actual.strip().split(",")]).astype(np.int64))
# act_products = []
# for pid in actual:
#     act_products.extend((df_products.loc[df_products.product_id == pid].product_name).tolist())
# print("Actual products bought by user {}\n{}\n\n".format(user_id, act_products))

# # All Products Recommended 
# all_recm_products=[]
# for recommend in recommendations_all:
#     all_recm_products.extend((df_products.loc[df_products.product_id == p_dict[recommend[0]]].product_name).tolist())
# print("All products recommended to user {}\n{}\n\n".format(user_id, all_recm_products))


# # New Products Recommended 
# new_recm_products=[]
# for recommend in recommendations_new:
#     new_recm_products.extend((df_products.loc[df_products.product_id == p_dict[recommend[0]]].product_name).tolist())
# print("New products recommended to user {}\n{}".format(user_id, new_recm_products))


# Evaluation

In [18]:
#Helper Functions
def get_k_popular(k, df_order_products_prior):
    popular_products = list(df_order_products_prior["product_id"].value_counts().head(k).index)
    return popular_products

In [19]:
# Transpose of the product_user utility matrix
user_product_matrix = product_user_matrix.T.tocsr()

# Number of recommendations to make for every user
N_REC = rec_items

# Get the `N_REC` most popular products
popular_products = get_k_popular(N_REC, df_order_products_prior)
print(popular_products)

[79, 5326, 889, 193, 601, 1010, 848, 761, 1919, 922]


In [20]:
def recall_score(actual, pred):
    """
    Given two lists representing actual and predicted values
    Returns the recall of the prediction
    """
    if len(actual) == 0:
        return 0
    actual, pred = set(actual), set(pred)
    return len(actual.intersection(pred)) / len(actual)


def new_products(row):
    """
    Given a row in the test dataset
    Returns the list of new products purchased
    """
    actual = row["products"][1:-1]  # Products purchased currently 
    actual = set([int(p.strip()) for p in actual.strip().split(",")])
    liked = set([p_dict[i] for i in user_product_matrix[u_dict[row["user_id"]]].indices])  # User's purchase history
    return actual - liked  # Return only new products purchased


def popular_recommend(row):
    """
    Given a row in the test dataset
    Returns the recall score when popular products are recommended
    """
    actual = new_products(row)
    return recall_score(actual, popular_products)

             
def svd_recommend_50_new(row):
    """
    Given a row in the test dataset
    Returns the recall score when our model recommends new products
    """    
    actual = new_products(row)
    recommended = svd_recm.recommend_new(u_dict[row["user_id"]], N=N_REC)
    recommended = [p_dict[r[0]] for r in recommended]
    return recall_score(actual, recommended)

def svd_recommend_100_new(row):
    """
    Given a row in the test dataset
    Returns the recall score when our model recommends new products
    """    
    actual = new_products(row)
    recommended = svd_recm_100.recommend_new(u_dict[row["user_id"]], N=N_REC)
    recommended = [p_dict[r[0]] for r in recommended]
    return recall_score(actual, recommended)


             
def build_eval_df(df_user_products_test, filepath=None, subset=None):
    """
    Builds a dataframe of recall values of the baseline and our model for all the users
    in the test data, and saves its to disk at `filepath`
    """
    start = time.time()
    print("Building dataframe with recall values ...")
    
    df_eval = df_user_products_test.copy()
    if subset:
        df_eval = df_eval.sample(n=int(len(df_eval) * subset), random_state=7)
    df_eval["popular_score"] = df_eval.apply(popular_recommend, axis=1)
    df_eval["svd_new_score_50"] = df_eval.apply(svd_recommend_50_new, axis=1)
    df_eval["svd_new_score_100"] = df_eval.apply(svd_recommend_100_new, axis=1)
    df_eval.to_csv(filepath, index=False)
    
    print("Completed in {:.2f}s".format(time.time() - start))    



In [21]:

# Get the dataframe with recall values of the baseline and the model
REBUILD_EVAL_DF = True
subset = 0.2  # Evaluate on `subset x 100`% of the test dataset
eval_path = "../data/eval/eval_discovery_svd_{}_{}.csv".format(subset if subset is not None else "full", N_REC)
if REBUILD_EVAL_DF or not Path(eval_path).exists():
    build_eval_df(df_user_products_test, filepath=eval_path, subset=subset)
df_eval = pd.read_csv(eval_path)

Building dataframe with recall values ...
Completed in 124.48s


# Results

In [22]:
# Mean recall scores
model_50_mean_recall,model_100_mean_recall, baseline_mean_recall = \
np.mean(df_eval["svd_new_score_50"]),np.mean(df_eval["svd_new_score_100"]), np.mean(df_eval["popular_score"])
print("SVD 100 Factor Model: {:.2f}%".format(model_100_mean_recall * 100))
print("SVD 50 Factor Model: {:.2f}%".format(model_50_mean_recall * 100))
print("Baseline: {:.2f}%".format(baseline_mean_recall * 100))

SVD 100 Factor Model: 1.31%
SVD 50 Factor Model: 1.08%
Baseline: 0.66%
